In [1]:
import numpy as np
import torch as th
torch = th
import time

# DPF v2 (PyTorch Version)
https://docs.google.com/presentation/d/108V1QMm7ACiD_mr8ITniYfQUwURSafZep6UJaQASZ0k/edit?folder=0AIi9WbrMGj2CUk9PVA#slide=id.g7643f50265_0_0

In [2]:
λ = 6 #6 #63 # security paramater
n = 8 #8 #32 # nb of bits of the algebric field

In [3]:
# PRG
def G(seed):
    #OK
    assert len(seed) == λ
    th.manual_seed(Convert(seed))
    return th.randint(2, size=(2*(λ + 1),), dtype=th.uint8)

bit_pow_lambda = th.flip(2 ** th.arange(λ), (0,)).to(th.uint8)
def Convert(bits):
    #OK
    return bits.dot(bit_pow_lambda)

def Array(*shape):
    #FIXED SHAPE
    return th.empty(shape, dtype=th.uint8)

bit_pow_n = th.flip(2 ** th.arange(n), (0,))
def bit_decomposition(x):
    return ((x & bit_pow_n) > 0).to(th.int8)

def randbit(size):
    #FIXED SHAPE
    return th.randint(2, size=size)
    
def concat(*args, **kwargs):
    return th.cat(args, **kwargs)

def split(x, idx):
    return th.split(x, idx)
    
one = th.tensor([1], dtype=th.uint8)
def TruthTableDPF(s, α_i):
    Table = th.zeros((2, λ+1), dtype=th.uint8)
    Table[α_i] = concat(s, one)
    return Table.flatten()

In [4]:
def GenDPF(alpha, beta):
    α = bit_decomposition(alpha)
    s, t, CW = Array(n+1, 2, λ), Array(n+1, 2), Array(n, 2*(λ+1))
    s[0] = randbit(size=(2, λ))
    t[0] = th.tensor([0, 1], dtype=torch.uint8)
    for i in range(0, n):
        # Re-use useless randomness
        sL_0, _, sR_0, _ = split(G(s[i, 0]), [λ, 1, λ, 1])
        sL_1, _, sR_1, _ = split(G(s[i, 1]), [λ, 1, λ, 1])
        s_rand = (sL_0 ^ sL_1) * α[i] + (sR_0 ^ sR_1) * (1 - α[i])
        
        cw_i = TruthTableDPF(s_rand, α[i])
        CW[i] = cw_i ^ G(s[i, 0]) ^ G(s[i, 1])
        
        for b in (0, 1):
            τ = G(s[i, b]) ^ (t[i, b] * CW[i])
            τ = τ.reshape(2, λ+1)
            s[i+1, b], t[i+1, b] = split(τ[𝛼[i]], [λ, 1])
        
    CW_n = (-1)**t[n, 1]*(beta.to(th.uint8) - Convert(s[n, 0]) + Convert(s[n, 1]))
    
    k = [(s[0, b], *CW, CW_n) for b in (0, 1)]
    return k
        

In [5]:

def EvalDPF(b, k_b, x):
    x = bit_decomposition(x)
    s, t = Array(n+1, λ), Array(n+1, 1)
    s[0], *CW = k_b
    t[0] = b
    for i in range(0, n):
        τ = G(s[i]) ^ (t[i]*CW[i])
        τ = τ.reshape(2, λ+1)
        s[i+1], t[i+1] = split(τ[x[i]], [λ, 1])
    return (-1)**b * (Convert(s[n]) + t[n]*CW[n])
        

In [6]:
def test_DPF():
    t = time.time()
    n_alpha, n_x = 10, 10
    beta = th.IntTensor([2])
    for alpha in th.randint(0, 2**n, (n_alpha, )):
        k = GenDPF(alpha, beta)
        t2 = time.time()
        for x in th.randint(0, 2**n, (n_x,)):
            y0 = EvalDPF(0, k[0], x)
            y1 = EvalDPF(1, k[1], x)
            
            #print(y0+y1)
            if x == alpha:
                assert y0+y1 == beta
            else:
                assert y0+y1 == 0
              
        #print((EvalDPF(0, k[0], alpha) + EvalDPF(1, k[1], alpha)))
        assert (EvalDPF(0, k[0], alpha) + EvalDPF(1, k[1], alpha)) == beta
        
        print(round(1000*(time.time() - t2)/((n_x+1) * 2), 1), 'ms / Eval')
    print(round(1000*(time.time() - t)/(n_alpha*n_x), 1), 'ms')
                
test_DPF()

2.7 ms / Eval
2.6 ms / Eval
2.5 ms / Eval
2.4 ms / Eval
2.4 ms / Eval
2.3 ms / Eval
2.3 ms / Eval
2.4 ms / Eval
2.6 ms / Eval
2.4 ms / Eval
6.9 ms


In [7]:
stop

NameError: name 'stop' is not defined

# DIF v3 simplified
We address here the problem $x \le \alpha$

https://docs.google.com/presentation/d/108V1QMm7ACiD_mr8ITniYfQUwURSafZep6UJaQASZ0k/edit?folder=0AIi9WbrMGj2CUk9PVA#slide=id.g6d24e8b0e8_0_1223

In [ ]:
def G(seed):
    assert len(seed) == λ
    r = np.random.RandomState(seed)
    return r.randint(2, size=2 + 2*(λ + 1))
    
    
def TruthTableDIF(s, α_i):
    leafTable = np.zeros((2, 1), dtype=np.int32)
    # if α_i is 0, then ending on the leaf branch means your bit is 1 to you're > α so you should get 0
    # if α_i is 1, then ending on the leaf branch means your bit is 0 to you're < α so you should get 1
    leaf_value = α_i
    leafTable[1-α_i] = leaf_value
    
    nextTable = np.zeros((2, λ+1), dtype=np.int32)
    nextTable[α_i] = concat(s, [1])
    
    #return concat(leafTable.flatten(), nextTable.flatten())
    return concat(leafTable, nextTable, axis=1).flatten()

In [ ]:
def GenDIF(alpha):
    α = bit_decomposition(alpha, nbits=n)
    s, t, CW = Array(n+1, 2, λ), Array(n+1, 2), Array(n, 2 + 2*(λ+1))
    s[0] = randbit(size=(2, λ))
    t[0] = [0, 1]
    for i in range(0, n):
        # Re-use useless randomness
        _, _, sL_0, _, sR_0, _ = split(G(s[i][0]), [1, 1, λ, 1, λ, 1])
        _, _, sL_1, _, sR_1, _ = split(G(s[i][1]), [1, 1, λ, 1, λ, 1])
        s_rand = xor(sL_0, sL_1) if α[i] else xor(sR_0, sR_1)
        cw_i = TruthTableDIF(s_rand, α[i])
        CW[i] = xor(cw_i, G(s[i][0]), G(s[i][1]))
        
        for b in (0, 1):
            τ = xor(G(s[i][b]), t[i][b] * CW[i])
            τ = τ.reshape(2, λ+2)
            σ_leaf, *s[i+1][b], t[i+1][b] = τ[𝛼[i]]
        
    k = [(s[0][b], CW) for b in (0, 1)]
    return k

In [ ]:
def EvalDIF(b, k_b, x, n):
    FnOutput = Array(n+1, 1)
    x = bit_decomposition(x, nbits=n)
    s, t = Array(n+1, λ), Array(n+1, 1)
    s[0], CW = k_b
    t[0] = b
    for i in range(0, n):
        τ = xor(G(s[i]), t[i]*CW[i])
        τ = τ.reshape(2, λ+2)
        σ_leaf, *s[i+1], t[i+1] = τ[x[i]]
        FnOutput[i] = σ_leaf

    # Last tour, the other σ is also a leaf:
    FnOutput[n] = t[n]
    return FnOutput.sum() % 2

In [ ]:
def test_DIF_simplified():
    t = time.time()
    n_alpha, n_x = 10, 10
    for alpha in np.random.randint(0,2**n, n_alpha):
        k = GenDIF(alpha)
        print('Test x <= α', alpha)
        t2 = time.time()
        for x in np.random.randint(0,2**n, n_x):
            sigma = xor(EvalDIF(0, k[0], x, n=n), EvalDIF(1, k[1], x, n=n))
            #print(f'x={x}', sigma)
            assert int(x<=alpha) == sigma
        print(round(1000*(time.time() - t2)/(n_x * 2), 1), 'ms / Eval')
    print(round(1000*(time.time() - t)/(n_alpha*n_x), 1), 'ms')

test_DIF_simplified()

# PyTorch

In [1]:
import syft as sy
import torch as th
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import time
import torch.autograd as autograd
import numpy as np

syft = sy 

hook = sy.TorchHook(th)
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
charlie = sy.VirtualWorker(hook, id="charlie")
james = sy.VirtualWorker(hook, id="james")
crypto_provider = james
torch = th

_get_hooked_func torch manual_seed torch.manual_seed


In [2]:
syft.hook.trace, syft.hook.trace_inactive = True, True

In [4]:
th.manual_seed(1)

torch.manual_seed
torch.manual_seed


In [5]:
th.random.manual_seed(1) #

## Equal zero

In [3]:
def G(seed):
    assert len(seed) == λ
    th.manual_seed(Convert(seed))
    return th.randint(2, size=(2*(λ + 1),), dtype=th.uint8)

In [4]:
field = 2**8
field_bounds = -2**7, 2**7-1
print('range', field_bounds)

def mod(x):
    if field_bounds[0] > x: 
        x += field
    elif x > field_bounds[1]:
        x -= field
    return x

def share(x):
    s = torch.randint(*field_bounds, x.shape)
    return mod(x-s).long(), s

range (-128, 127)


In [5]:
x = th.tensor([0.])
s = torch.randint(*field_bounds, (1,))
x_priv = share(x.fix_prec(base=2, precision_fractional=3, field=field).child.child)
_ = x_priv, sum(x_priv)

In [6]:
def keygen_equal_zero():
    r_in = torch.randint(*field_bounds, (1,))
    beta = th.LongTensor([1])
    k = GenDPF(r_in, beta)
    return share(r_in), k

In [14]:
def equal_zero(x_priv):
    r, k = keygen_equal_zero()
    share_0 = x_priv[0] + r[0]
    share_1 = x_priv[1] + r[1]
    x_pub = mod(share_0 + share_1)
    res_0 = EvalDPF(0, k[0], x_pub)
    res_1 = EvalDPF(1, k[1], x_pub)
    return res_0, res_1

In [15]:
sy.hook.trace = True
sy.hook.trace_inactive = True

In [16]:
# Disclose value
res_0, res_1 = equal_zero(x_priv)
#print(mod(res_0 + res_1))

torch.randint
__and__
__gt__
to
tolist
torch.empty
torch.empty
torch.empty
torch.randint
__setitem__
__setitem__
__getitem__
dot
torch.random.manual_seed


AttributeError: module 'torch.random' has no attribute 'native_manual_seed'

In [17]:
th.manual_seed(1)

In [ ]:
r_in = torch.randint(*field_bounds, (1,))
r_in

In [ ]:
beta = th.LongTensor([1])
k = GenDPF(r_in, beta)
eval_trace = torch.jit.trace(EvalDPF,(th.LongTensor([1]), k[1], r_in))

In [ ]:
mod(eval_trace(th.LongTensor([0]), k[0], r_in) + eval_trace(th.LongTensor([1]), k[1], r_in))

In [ ]:
mod(EvalDPF(th.LongTensor([0]), k[0], r_in) + EvalDPF(th.LongTensor([1]), k[1], r_in))

In [ ]:
r_in = torch.randint(*field_bounds, (1,))
beta = th.LongTensor([1])
GenDPF(r_in, beta)

In [ ]:
p = GenDPF.send(bob)